# Estimación probabilística de la dosis de exposición dérmica


## Estimación de la concentración de la sustancia de interés en la solución a considerar

In [1]:
import numpy as np 
import pandas as pd
import random
import matplotlib.pyplot as plt
from scipy.stats import lognorm
from scipy.stats import iqr
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
%matplotlib inline 



dtcnc = []

# Define the function to be plotted
def plot_function(mean, stdev):
    global dtcnc
    phi = (stdev ** 2 + mean ** 2) ** 0.5
    mu = np.log(mean ** 2 / phi)
    sigma = (np.log(phi ** 2 / mean ** 2)) ** 0.5
    dtcnc = np.random.lognormal(mu, sigma , 5000)
    # mu, sigma, n= lognorm.fit(data)
    plt.figure(figsize=(10,8))
    plt.hist(dtcnc, bins=30, density=True, alpha=0.5, color="b")
    # Plot the PDF.
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 1000)
    p = lognorm.pdf(x = x, scale = mean, s = sigma)
    plt.plot(x, p, 'k', linewidth=2)
    plt.ylabel('Densidad de probabilidad')
    plt.xlabel('mg/cm$^{3}$', y=0.05)
    plt.savefig('log_normal_sln.png', bbox_inches='tight', dpi=300)
    plt.show()

# Define the slider widgets
slider_mean = widgets.FloatSlider(min=0, max=10, step=0.01, value=0.07)
slider_stdev = widgets.FloatSlider(min=0, max=10, step=0.01, value=0.07*(1.125/1.07))

# Create the interactive plot
interactive_plot = widgets.interactive(plot_function, mean=slider_mean, stdev=slider_stdev)
display(interactive_plot)
dtcnc = interactive_plot.result

interactive(children=(FloatSlider(value=0.07, description='mean', max=10.0, step=0.01), FloatSlider(value=0.07…

## Estimación del coeficiente de permeabilidad dérmica

*Ecuación extraida del Risk assessment guidance for superfund volume I: Human health evaluaction manual (Part E, supplemental guidance for dermal risk assessment. A.1.1 Estimation of Kp for organic compounds)*

$$ 
\begin{align}
  log(Kp) = -2.80+(0.66*log(Kw)-(0.0056*mw)
\end{align} 
$$
$$
\begin{align}
  C_{permeabilidad} = 10^{log(Kp)}
\end{align} 
$$

In [2]:
def lgkp(lgkw, mw):
    logKp = -2.80+(0.66*lgkw)-(0.0056*mw)
    prmcf = 10**(logKp)
    return prmcf

slider_lgkw1 = widgets.FloatSlider(description='lgkw', min=-10, step=0.1, value=3.4)
slider_mw1 = widgets.FloatSlider(description='mw', min=0, max=10000, step=0.1, value=650.7)
out = widgets.Output()

def update(change):
    with out:
        out.clear_output()
        prmcf = lgkp(slider_lgkw1.value, slider_mw1.value)
        print("Coeficiente de permeabilidad dérmica de la sustancia:", prmcf)

slider_lgkw1.observe(update, 'value')
slider_mw1.observe(update, 'value')

display(widgets.VBox([slider_lgkw1, slider_mw1, out]))

In [3]:
areakde = pd.read_csv('../28022023/areakde.csv').loc[:, '0'].values

In [4]:
pesokde = pd.read_csv('../28022023/pesokde.csv').loc[:, '0'].values

## Resultados de la estimación probablística de las dosis de exposición mediante el uso del método de Monte Carlo

*Distribución de la concentración de la sustancia de interés en la solución a considerar; distribución estadística de las dosis de exposición y diagrama de cajas (boxplot) de esas dosis de exposición. Resaltado en rojo se encuentra el percentil 97.5 de la dosis de exposición*

In [5]:
def hstgrm(xpdrm, bins, xlim):
    plt.figure(figsize=(10,8))
    plt.xlim(xlim)
    plt.hist(xpdrm, density=True, bins=bins)  
    plt.ylabel('Densidad de probabilidad')
    plt.xlabel('mg/kg·día');
    plt.axvline(x = np.percentile(xpdrm, 50), linewidth=1, color='r', linestyle=':')
    plt.axvline(x = np.percentile(xpdrm, 2.5), linewidth=1, color='r', linestyle='--')
    plt.axvline(x = np.percentile(xpdrm, 97.5), linewidth=1, color='r', linestyle='dashed')
    plt.text((np.percentile(xpdrm, 97.5)+0.00005*iqr(xpdrm)), len(xpdrm)/sum(xpdrm), 'P97.5 = '+np.format_float_scientific(np.percentile(xpdrm, 97.5), precision=2), rotation=90, color='red', verticalalignment='center')
    plt.savefig('exporal_dermica.png', bbox_inches='tight', dpi=300)
    plt.show()
    
def montecarlo(prcrdrmxpt, prcbsrdrm, dtcnc, lgkw, mw, tm, N):
    global prmcf
    random_concsolap = np.random.choice(dtcnc, size=N)
    random_ardrmc = np.random.choice(areakde, size=N)
    random_peso = np.random.choice(pesokde, size=N)
    prmcf = lgkp(lgkw, mw)
    expderm = (((random_concsolap*random_ardrmc*prcrdrmxpt)*(prcbsrdrm*prmcf*tm))/random_peso)
    return expderm

def hstgrm_wrapper(prcrdrmxpt, prcbsrdrm, tm, bins):
    global dtcnc, prmcf
    xpdrm = montecarlo(prcrdrmxpt, prcbsrdrm, dtcnc, slider_lgkw.value, slider_mw.value, tm, 10000)
    xlim = (0, np.percentile(xpdrm, 97.5)+iqr(xpdrm)+(0.01*iqr(xpdrm)))
    plt.clf()
    plt.figure(figsize=(10,8))
    hstgrm(xpdrm, bins, xlim)
    plt.figure(figsize=(10,8))
    plt.boxplot(np.array(xpdrm)[np.array(xpdrm < np.percentile(xpdrm, 97.5)+iqr(xpdrm))], vert=False, patch_artist=True)
    plt.xlabel('mg/kg·día')
    plt.yticks([1], [''])
    plt.axvline(x = np.quantile(xpdrm, 0.975) , linewidth=1, color='r', linestyle='--')
    plt.text((np.percentile(xpdrm, 97.5)+0.000022), 1.125, 'P97.5 = '+np.format_float_scientific(np.percentile(xpdrm, 97.5), precision=2), color='red', rotation=90)
    plt.xlim(xlim)
    plt.savefig('boxplot_dermica.png', bbox_inches='tight', dpi=300)
    plt.show()
    
slider_mean = widgets.FloatSlider(min=0, max=10, step=0.01, value=0.07)
slider_stdev = widgets.FloatSlider(min=0, max=10, step=0.01, value=0.07*(1.125/1.07))
slider_prcrdrmxpt = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.33)
slider_prcbsrdrm = widgets.FloatText(min=0.000001, max=1.0, step=0.01, value=0.039)
slider_lgkw = widgets.FloatSlider(description='lgkw', min=-10, step=0.1, value=3.4)
slider_mw = widgets.FloatSlider(description='mw', min=0, max=10000, step=0.1, value=650.7)
slider_tm = widgets.FloatSlider(min=0, max=24, value=24)
slider_bins = widgets.IntSlider(min=1, max=10000, step=1, value=1000)

def interactive_plot_func(mean, stdev, prcrdrmxpt, prcbsrdrm, lgkw, mw, tm, bins):
    plot_function(mean, stdev);
    hstgrm_wrapper(prcrdrmxpt, prcbsrdrm, tm, bins);

interactive_plot = widgets.interactive(interactive_plot_func, 
                                       mean=slider_mean, 
                                       stdev=slider_stdev,
                                       prcrdrmxpt=slider_prcrdrmxpt,
                                       prcbsrdrm=slider_prcbsrdrm,
                                       lgkw=slider_lgkw,
                                       mw=slider_mw,
                                       tm=slider_tm,
                                       bins=slider_bins)
display(interactive_plot)

interactive(children=(FloatSlider(value=0.07, description='mean', max=10.0, step=0.01), FloatSlider(value=0.07…